In [2]:
import numpy as np
from dipy.segment.clustering import QuickBundles
from dipy.io.streamline import load_trk, save_trk
from dipy.segment.metric import ResampleFeature, AveragePointwiseEuclideanMetric,mdf
from dipy.io.image import load_nifti
import warnings
from dipy.viz import window, actor
from time import sleep
from dipy.tracking.streamline import set_number_of_points
from dipy.tracking.streamline import transform_streamlines
import os, glob
from tract_save import unload_trk
import pickle
from dipy.tracking.utils import connectivity_matrix
from nifti_handler import getlabeltypemask
from file_tools import mkcdir
from tract_handler import ratio_to_str, gettrkpath
from convert_atlas_mask import convert_labelmask, atlas_converter
import errno

def show_bundles(bundles, colors=None, show=True, fname=None, str_tube=False, ref=None):
    ren = window.Renderer()
    ren.SetBackground(1., 1, 1)
    if str_tube:
        bundle_actor = actor.streamtube(bundles, colors, linewidth=0.5)
        ren.add(bundle_actor)
    else:
        for (i, bundle) in enumerate(bundles):
            color = colors[i]
            #         lines_actor = actor.streamtube(bundle, color, linewidth=0.05

            lines_actor = actor.line(bundle, color, linewidth=2.5)
            # lines_actor.RotateX(-90)
            # lines_actor.RotateZ(90)
            ren.add(lines_actor)

    if ref is not None:
        if os.path.exists(ref):
            data, affine = load_nifti(ref)
            #data, affine = load_nifti('/Volumes/Data/Badea/Lab/mouse/VBM_19IntractEP01_IITmean_RPI-work/dwi/SyN_0p5_3_0p5_dwi/dwiMDT_NoNameYet_n7_i6/median_images/MDT_fa.nii.gz')
            ref_actor = actor.slicer(data, affine)
            ren.add(ref_actor)
        else:
            txt = f'Was asked to find reference file for background at {ref} but path did not exist'
            warnings.warn(txt)

    if show:
        window.show(ren)
    if fname is not None:
        sleep(1)
        window.record(ren, n_frames=1, out_path=fname, size=(900, 900))

def get_grouping(grouping_xlsx):
    print('not done yet')

def get_diff_ref(label_folder, subject, ref):
    diff_path = os.path.join(label_folder,f'{subject}_{ref}_to_MDT.nii.gz')
    if os.path.exists(diff_path):
        return diff_path
    else:
        raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), diff_path)

ModuleNotFoundError: No module named 'tract_save'

In [ ]:
## AMD parameters

#set parameter
num_points1 = 50
distance1 = 1
feature1 = ResampleFeature(nb_points=num_points1)
metric1 = AveragePointwiseEuclideanMetric(feature=feature1)

#group cluster parameter
num_points2 = 50
distance2 = 2
feature2 = ResampleFeature(nb_points=num_points2)
metric2 = AveragePointwiseEuclideanMetric(feature=feature2)


samos = False
if samos:
    TRK_folder = '/mnt/paros_MRI/jacques/AMD/TRK_MDT_fixed'
    label_folder = '/mnt/paros_MRI/jacques/AMD/DWI'
    trkpaths = glob.glob(os.path.join(TRK_folder,'*trk'))
    figures_folder = '/mnt/paros_MRI/jacques/AMD/Figures_MDT'

santorini = True
if santorini:
    TRK_folder = '/Users/alex/jacques/AMD_testing_zone/TRK_MDT'
    label_folder = '/Users/alex/jacques/AMD_testing_zone/DWI'
    trkpaths = glob.glob(os.path.join(TRK_folder,'*trk'))
    figures_folder = '/Users/alex/jacques/AMD_testing_zone/Figures_MDT'
    pickle_folder = '/Users/alex/jacques/AMD_testing_zone/Pickle_MDT'
    centroid_folder = '/Users/alex/jacques/AMD_testing_zone/Centroids_MDT'
    mkcdir([figures_folder, pickle_folder, centroid_folder])
    if not os.path.exists(TRK_folder):
        raise Exception(f'cannot find TRK folder at {TRK_folder}')
    ROI_legends = "/Volumes/Data/Badea/ADdecode.01/Analysis/atlases/IITmean_RPI/IITmean_RPI_index.xlsx"